## Ingest data from race.csv
##### TODO:
1. Read the data from `race.csv` file into a dataframe using DataframeReader
2. Select only the columns except `url` column
3. Rename the columns as required 
- raceId to race_id
- year to racce_year
- circuitId to circuit_id

4. add new column race_timestamp by transforming date and time columns
5. add a new column to our existing dataframe (ingestion_date) 

5. Write data into a parquet file using - DataframeWriter

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
'''
raceId: string (nullable = true)
 |-- year: string (nullable = true)
 |-- round: string (nullable = true)
 |-- circuitId: string (nullable = true)
 |-- name: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- url: string (nullable = true)
 '''

race_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                 StructField("year", IntegerType(), True),
                                 StructField("round", IntegerType(), True),
                                 StructField("circuitId", IntegerType(), True),
                                 StructField("name", StringType(), True),
                                 StructField("date", DateType(), True),
                                 StructField("time", StringType(), True),
                                 StructField("url", StringType(), True)

])


In [0]:
race_raw_df = spark.read \
    .option("header", True) \
    .schema(race_schema) \
    .csv('/Volumes/formula1/default/f1_volume/raw/races.csv')

In [0]:
race_raw_df.printSchema()

In [0]:
from pyspark.sql.functions import current_timestamp, to_timestamp, concat, col, lit

In [0]:
race_new_columns = race_raw_df.withColumn("ingestion_date", current_timestamp()) \
.withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss'))
                                        

In [0]:
display(race_new_columns)

In [0]:
race_selected_df = race_new_columns.select(col("raceId").alias("race_id"), col("year").alias("race_year"), col("round"), col("circuitId").alias("circuit_id"), col("name"), col("race_timestamp"), col("ingestion_date"))

In [0]:
display(race_selected_df)

In [0]:
race_selected_df.write.mode('overwrite').parquet('/mnt/formula19533dl/processed/races')